In [ ]:
import cv2
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
################# Initialize the paths #################

input_dir = "C:\\Users\\DELL\\Documents\\DL_Assignment\\CRNN\\Raw"  

output_drowsy_dir = "C:\\Users\\DELL\\Documents\\DL_Assignment\\CRNN\\Augment\\Drowsy"
output_non_drowsy_dir = "C:\\Users\\DELL\\Documents\\DL_Assignment\\CRNN\\Augment\\Non Drowsy"
output_dir_resize = "C:\\Users\\DELL\\Documents\\DL_Assignment\\CRNN\\Resize"
parent_dir = "C:\\Users\\DELL\\Documents\\DL_Assignment\\CRNN\\Augment"
os.makedirs(output_drowsy_dir, exist_ok=True)
os.makedirs(output_non_drowsy_dir, exist_ok=True)
os.makedirs(output_dir_resize, exist_ok=True)
classes = ["Drowsy", "Non Drowsy"]

In [ ]:
################ Data Augmentation ###################

rotation_angles = [-10, 10 ]  

# Perform data augmentation for each class
for class_name in classes:
    class_input_dir = os.path.join(input_dir, class_name)
    class_output_dir = output_drowsy_dir if class_name == "Drowsy" else output_non_drowsy_dir

    # List the files in the class input directory
    file_list = os.listdir(class_input_dir)

    for file_name in file_list:
        if file_name.endswith(".png"):  
            # Read the image
            img = cv2.imread(os.path.join(class_input_dir, file_name))

           
            for angle in rotation_angles:
                # Rotate the image
                rotated_img = np.array(Image.fromarray(img).rotate(angle))

                # Save the augmented image in the appropriate class output directory
                output_file = os.path.join(class_output_dir, f"rotated_{angle}_{file_name}")
                cv2.imwrite(output_file, rotated_img)

            # Horizontal flip
            flipped_img = cv2.flip(img, 1)

            # Save the flipped image in the appropriate class output directory
            output_file = os.path.join(class_output_dir, f"flipped_{file_name}")
            cv2.imwrite(output_file, flipped_img)

In [ ]:
######################## Resize Images #########################

target_size = (227, 227)

for class_name in classes:
    class_dir = os.path.join(parent_dir, class_name)
    output_class_dir = os.path.join(output_dir_resize, class_name)
    os.makedirs(output_class_dir, exist_ok=True)

    # List the files in the class directory
    file_list = os.listdir(class_dir)

    # Resize and pad each image
    for file_name in file_list:
        if file_name.endswith(".png"):
            # Open the image
            img = Image.open(os.path.join(class_dir, file_name))

            # Resize the image while maintaining its aspect ratio
            img.thumbnail(target_size, Image.ANTIALIAS)

            # Create a new image with the target size and a white background
            new_img = Image.new("RGB", target_size, (255, 255, 255))

            # Calculate the position to center the resized image
            left = (target_size[0] - img.width) // 2
            top = (target_size[1] - img.height) // 2

            # Paste the resized image onto the new image with padding
            new_img.paste(img, (left, top))

            # Save the resized and padded image in the output directory
            output_file = os.path.join(output_class_dir, file_name)
            new_img.save(output_file)

print("Resizing and padding complete.")

In [ ]:
################ Calculate Mean and Standard deviation for z-score normalization ######################

parent_dir = "C:\\Users\\DELL\\Documents\\DL_Assignment\\CRNN\\Resize"
mean_accumulator = np.zeros((3,))
stddev_accumulator = np.zeros((3,))
total_images = 0

for class_name in classes:
    class_dir = os.path.join(parent_dir, class_name)

    # List the files in the class directory
    file_list = os.listdir(class_dir)

    for file_name in file_list:
        if file_name.endswith(".png"): 
            # Read the image
            img = cv2.imread(os.path.join(class_dir, file_name))

            # Convert to float and divide by 255 for values in [0, 1]
            img_float = img.astype(np.float32) / 255.0

            # Accumulate pixel values
            mean_accumulator += np.mean(img_float, axis=(0, 1))
            stddev_accumulator += np.std(img_float, axis=(0, 1))
            total_images += 1

# Calculate the overall mean and stddev
overall_mean = mean_accumulator / total_images
overall_stddev = stddev_accumulator / total_images

print("Overall Mean:", overall_mean)
print("Overall Standard Deviation (stddev):", overall_stddev)

In [ ]:
########## Apply z-score normalization #################

out_dir_nor = "C:\\Users\\DELL\\Documents\\DL_Assignment\\CRNN\\Normalize"
def z_score_normalization(img):
    # Convert to float and divide by 255 for values in [0, 1]
    img_float = img.astype(np.float32) / 255.0

    # Apply mean subtraction and standard deviation scaling (z-score normalization)
    normalized_img = (img_float - overall_mean) / overall_stddev

    # Ensure the pixel values are in the correct range [0, 1]
    normalized_img = np.clip(normalized_img, 0, 1)

    return normalized_img

# Iterate over all images and apply normalization
for class_name in classes:
    class_dir = os.path.join(parent_dir, class_name)
    output_dir = os.path.join(out_dir_nor, f"{class_name}")
    os.makedirs(output_dir, exist_ok=True)

    # List the files in the class directory
    file_list = os.listdir(class_dir)

    for file_name in file_list:
        if file_name.endswith(".png"):  # Adjust the file extension as needed
            # Read the image
            img = cv2.imread(os.path.join(class_dir, file_name))

            # Apply z-score normalization
            normalized_img = z_score_normalization(img)

            # Save the normalized image
            output_file = os.path.join(output_dir, file_name)
            cv2.imwrite(output_file, (normalized_img * 255).astype(np.uint8))

print("Z-score normalization complete.")